In [241]:
import pandas as pd
import re
import requests
import difflib
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [263]:
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get('https://fantasyfootballcalculator.com/adp', headers=header)
ffc = pd.read_html(r.text, header = 0); ffc = ffc[0]

ffc = ffc[['Pick', 'Name', 'Overall', 'Std.Dev', 'High', 'Low']]
ffc = ffc.rename(columns = {'Name': 'name'})



fp = pd.read_html('https://www.fantasypros.com/nfl/adp/overall')[0]
fp['Player Team (Bye)'] = fp.iloc[:,1].astype('str')
name = []
for i in range(0,len(fp)):
    try: 
        name.append (re.findall('(.*?)\s[A-Z][A-Z]', fp.iloc[i,1])[0])
    except:
        name.append("")
    

fp['name'] = name
fp = fp.drop('Player Team (Bye)', axis = 1)

stopwords = [" Jr.", " II", " III", " IV", '\'', "Defense"]



fp['name'].apply(lambda x: [item for item in x if item not in stopwords])

pat = r'\b(?:{})\b'.format('|'.join(stopwords))
fp['name'] = fp['name'].str.replace(pat, '')
ffc['name'] = ffc['name'].str.replace(pat, '')
fp['name'] = fp['name'].str.replace('.', '')
ffc['name'] = ffc['name'].str.replace('.', '')


fp['name'] = fp['name'].str.upper()
ffc['name'] = ffc['name'].str.upper()

rem_name = []
remainder = ffc[-ffc['name'].isin(fp['name'])]
for i in range(0, len(remainder)):
    rem_name.append( difflib.get_close_matches(remainder.iloc[i,1], fp['name'], cutoff = .55)[0])

indexer = ffc[-ffc['name'].isin(fp['name'])].index
ffc.loc[indexer, "name"] = rem_name



df = ffc.merge(fp)

df.to_csv('../adp.csv')


In [221]:
len(df)

197

In [226]:
df.sort_values('Rank')

,Pick,name,Overall,Std.Dev,High,Low,Rank,POS,CBS,FFC,Fantrax,AVG
0,1.02,SAQUON BARKLEY,1.5,0.7,1.01,1.04,1,RB1,1.0,1.0,1.0,1.0
1,1.02,ALVIN KAMARA,2.3,0.9,1.01,1.04,2,RB2,2.0,2.0,3.0,2.3
2,1.03,CHRISTIAN MCCAFFREY,2.5,0.9,1.01,1.05,3,RB3,3.0,3.0,2.0,2.7
4,1.05,EZEKIEL ELLIOTT,4.8,1.5,1.01,1.11,4,RB4,4.0,5.0,4.0,4.3
3,1.05,DAVID JOHNSON,4.8,1.1,1.02,1.11,5,RB5,7.0,4.0,5.0,5.3
5,1.06,DEANDRE HOPKINS,6.3,1.8,1.02,1.12,6,WR1,5.0,6.0,6.0,5.7
6,1.07,LEVEON BELL,7.4,2.2,1.01,2.05,7,RB6,6.0,7.0,7.0,6.7
7,1.08,DAVANTE ADAMS,8.1,1.9,1.01,2.02,8,WR2,9.0,8.0,8.0,8.3
9,1.10,JAMES CONNER,9.5,2.2,1.04,2.06,9,RB7,10.0,10.0,9.0,9.7
12,2.01,MICHAEL THOMAS,12.8,2.4,1.06,2.08,10,WR3,12.0,13.0,11.0,12.0


In [259]:
df['error'] =  100 * abs(((df['Overall'] - df['AVG']))/ df['AVG'])

In [260]:
df['error'].describe()

count    197.000000
mean       9.184469
std        7.722804
min        0.000000
25%        3.176342
50%        7.441860
75%       13.410771
max       50.000000
Name: error, dtype: float64

In [261]:
df.sort_values('error')

,Pick,name,Overall,Std.Dev,High,Low,Rank,POS,CBS,FFC,Fantrax,AVG,error
1,1.02,ALVIN KAMARA,2.3,0.9,1.01,1.04,2,RB2,2.0,2.0,3.0,2.3,0.000000
45,4.09,PHILLIP LINDSAY,45.3,5.5,2.10,5.11,45,RB23,48.0,46.0,42.0,45.3,0.000000
85,7.12,JORDAN HOWARD,83.7,8.4,4.05,9.04,81,RB35,90.0,86.0,75.0,83.7,0.000000
187,14.11,TREQUAN SMITH,166.9,12.5,10.09,15.12,182,WR63,NaN,188.0,146.0,167.0,0.059880
167,14.01,JORDAN REED,157.2,19.4,8.10,15.12,156,TE16,118.0,168.0,186.0,157.3,0.063573
52,5.05,CALVIN RIDLEY,53.4,4.8,4.01,6.12,54,WR23,54.0,53.0,53.0,53.3,0.187617
129,11.05,LAMAR JACKSON,124.6,9.8,7.12,13.05,120,QB16,122.0,130.0,121.0,124.3,0.241352
126,11.03,MATT BREIDA,122.6,12.2,7.11,13.07,118,RB45,124.0,127.0,118.0,123.0,0.325203
30,3.06,TY HILTON,29.8,4.6,2.02,4.07,29,WR12,30.0,31.0,28.0,29.7,0.336700
143,12.07,DAK PRESCOTT,138.5,13.0,9.07,15.09,141,QB18,146.0,144.0,127.0,139.0,0.359712


99